In [ ]:
# ! export http_proxy=http://172.22.160.1:7880
# ! export https_proxy=http://172.22.160.1:7880
# ! wget google.com

import os
import json
from poe_api_wrapper import PoeApi, PoeExample

os.environ["http_proxy"] = "http://172.22.160.1:7880"
os.environ["https_proxy"] = "http://172.22.160.1:7880"

api_config_file = "api_configs.json"
with open(api_config_file, "r") as f:
    api_configs = json.load(f)

tokens = api_configs["poe_tokens"]

In [ ]:
# print(tokens)
tokens = {
    "b": "NxeuhIbAINHu0OM49hkvWw%3D%3D",
    "lat": "4vIeYDWHpqB7nlwlXGAkQ8u5nAKFG98FjueVCPPDlw%3D%3D",
}
client = PoeApi(cookie=tokens)

In [ ]:
hist = client.get_chat_history()
hist["data"]['capybara'][0]

In [ ]:
ret=client.send_message("Assistant", "Hello, please write a 1000 word essay describing a beautiful house!")

In [ ]:
ans = []
for chunk in ret:
    pass
chunk

In [ ]:
ans

In [1]:
# imports
import os
import os.path as osp
import sys
import json
import tqdm
import yaml
import copy

from poe_api_wrapper import PoeApi
from cache import Cacher

# configs
os.environ["http_proxy"] = "http://172.22.160.1:7880"
os.environ["https_proxy"] = "http://172.22.160.1:7880"

api_config_file = "api_configs.json"
with open(api_config_file, "r") as f:
    api_configs = json.load(f)

tokens = api_configs["poe_tokens"]
tokens


# experiment settings

config_path = "results/configs"
config_name = "test"
config_path = osp.join(config_path, config_name + ".yaml")

configs = yaml.load(open(config_path, "r"), Loader=yaml.FullLoader)

dataset_path = osp.join(
    configs["data"]["data_path"], configs["data"]["dataset_name"] + ".json"
)

if configs["use_cache"]:
    cache_path = osp.join(
        configs["save_path"], "cache", "cache_" + configs["cache_name"] + ".json"
    )
else:
    cache_path = osp.join(configs["save_path"], "cache", "cache_default" + ".json")

    # initialize apis

client = PoeApi(tokens)
cacher = Cacher(
    cache_path,
    cache_id="default",
    cache_type="json",
    write_cache=True,
    write_interval=1,
)

print("APIs initialized, using cache at", cache_path)


# experiment

# load dataset
with open(dataset_path, "r") as f:
    dataset = json.load(f)

tot_price = 0

os.makedirs(
    osp.join(
        configs["save_path"], "responds", configs["data"]["dataset_name"], config_name
    ),
    exist_ok=True,
)

bot = configs["bot_list"][0]
answers = copy.deepcopy(dataset)

task_id, task = 0, dataset["tasks"][0]

task_text = task["text"]

task_text = (
    configs["text_modifier"]["prefixes"]
    + task_text
    + configs["text_modifier"]["suffixes"]
)

answers["tasks"][task_id]["prefixes"] = configs["text_modifier"]["prefixes"]
answers["tasks"][task_id]["suffixes"] = configs["text_modifier"]["suffixes"]
answers["tasks"][task_id]["answers"] = []



APIs initialized, using cache at results/cache/cache_default.json


In [5]:
message = task_text
message

'1) Think of a random number between 10 and 20. 2) Call that number X and tell me what it is. 3) Estimate whether John, a phone shop owner, has more or less than X phones in his shop. 4) Tell me the exact number of phones you think John has in his shop. Answer without justification.'

In [3]:
cacher.load_cache()

In [4]:
current_bot_cache = cacher.cache[bot]

In [6]:
message not in current_bot_cache

False

In [ ]:
ans = cacher.send_message(
    client=client,
    bot=bot,
    message=task_text,
    use_cache=True,
    add_to_cache=True,
    trial_id=0,
)